# 라이브러리

In [32]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 정의 함수

In [36]:
# 텍스트 전처리
def prepro_text(raw_data):

    # 특수기호, 영어, 숫자 제거
    prepro_texts = re.sub(r'[^가-힣]',' ',str(raw_data))

    # 형태소 분석기 생성
    okt = Okt()

    # 조사와 복수표현 등 필요없는 품사 tag 제거
    prepro_word = []
    for word, tag in okt.pos(prepro_texts):
        if tag not in ['Josa', 'Suffix']:
            prepro_word.append(word)
    
    # 어간 추출 적용
    # result = ' '.join(okt.morphs(' '.join(prepro_word), stem=True))

    result = ' '.join(prepro_word)

    return result


# 빈도분석 후 결과 출력
def count_analyze(texts, count_vec, color=None, title=None):
    
    count_vec.fit(texts)
    
    word_dict = sorted(count_vec.vocabulary_.items())
    idx2word = {idx:word for word, idx in word_dict}

    total_text = []
    total_text.append(' '.join(texts.values))

    count_matrix = count_vec.transform(total_text)

    count_word = []
    count_vector = []

    for i in range(20,0,-1):
        count_word.append(idx2word[(-count_matrix.toarray()[0]).argsort()[i-1]])
        count_vector.append(count_matrix.toarray()[0][(-count_matrix.toarray()[0]).argsort()[i-1]])

    # print(count_word)
    # print(count_vector)

    # plt.barh(count_word, count_vector, color=color)
    # plt.yticks(count_word)
    # plt.title(f'{title} 빈도 분석')
    # plt.show()

    return count_word, count_vector


# 코사인 유사도 score 추출
def cosine_extraction(vec, fit_data, count_words):

    # 적합 후 벡터 변화
    analysis_vec = vec.fit_transform(fit_data)    

    # 빈도분석으로 뽑은 Top10 단어들 하나의 text로
    top10_word_vec = vec.transform([' '.join(count_words)])

    cosine_sim = cosine_similarity(top10_word_vec, analysis_vec)

    cos_sim = fit_data.name

    cosine_df = pd.DataFrame({
        'id' : fit_data.index.values,
        cos_sim : cosine_sim[0]
    })
    #fit_cosine_df['코사인 유사도'] = fit_cosine_df['코사인 유사도'].map(lambda x: round(x,3))
    #fit_cosine_df['rank'] = fit_cosine_df['코사인 유사도'].rank(ascending=False).astype(np.int64)

    return cosine_df

# 데이터

In [2]:
path = './히든그레이스_자소서_텍스트분석_data.xlsx'
df = pd.read_excel(path)

In [3]:
for idx, title in enumerate(df.columns.values):
    if title == 'label2':
        print(idx, title)
    elif title == '히든그레이스는 어떤 회사인가':
        print(idx, title)
    elif title == '히든그레이스가 보완할 점':
        print(idx, title)
    elif title == '채용 된다면 일할 수 있는 기간, 오래 일하는 데 문제 되는 요소':
        print(idx, title)

17 label2
25 히든그레이스는 어떤 회사인가
26 히든그레이스가 보완할 점
27 채용 된다면 일할 수 있는 기간, 오래 일하는 데 문제 되는 요소


# 유저 등록

In [ ]:
# 유저 등록
url = 'http://localhost:3100/api/v1/'

for idx in df.index:
    first_name = df.iloc[idx,0][1:]
    last_name = df.iloc[idx,0][:1]
    data ={
        "firstName": first_name,
        "lastName": last_name,
        "email": f"dev{idx}@test.com",
        "phone": f"010-222-333{idx}",
        "company": "히든그레이스",
        "country": 1,
        "language": 1,
        "password": f"123456{idx}"
        }
    res = requests.post(url+'auth/register', data)
    print(res.status_code)

# 자소서 등록

In [ ]:
# 자소서 등록
for idx in df.index:
    label = df.iloc[idx,17]
    q_1 = df.iloc[idx, 25]
    q_2 = df.iloc[idx, 26]
    q_3 = df.iloc[idx, 27]
    data =[{
        "userId": idx+1,
        "label": int(label),
        "cosine": 0.01,
        "rank": 0,
        "questions": [
                {
                    "label": "Q1",
                    "data": q_1
                },
                {
                    "label": "Q2",
                    "data": q_2
                },
                {
                    "label": "Q3",
                    "data": q_3
                }
            ]
        }]
    header = {'Content-Type': 'application/json'}
    text_res = requests.put(url+'cosine/update', json.dumps(data), headers=header)
    print(text_res.status_code)

# 자소서 정보 가져 오기

In [7]:
# 예시
url = 'http://localhost:3100/api/v1/'
get_data ={
    'userId':1,
    'include':[
        'cosine'
    ]
}
header = {'Content-Type': 'application/json'}
get_res =requests.post(url+'user', json.dumps(get_data), headers=header)
print(get_res.json())

{'success': True, 'result': {'user': [{'userId': 1, 'firstName': '탁근', 'lastName': '오', 'email': 'dev0@test.com', 'phone': '010-222-3330', 'cosine': {'id': 1, 'label': 0, 'cosine': 0.01, 'rank': 0, 'questions': [{'id': 7, 'label': 'Q1', 'data': '재작년에 진행했던 장애청년드림팀을 통해 다양한 장애를 가진 팀원들을 만나보고 느낀 점은 특정 장애가 있다고 하여 그 부분이 불가능한 것은 아니라는 것이었습니다. 지체장애를 가진 팀원은 체계적은 훈련 하에 몇 일만에 성공적으로 스쿠버다이빙을 완수하게 되었고 시각장애를 가진 팀원은 다른 선진국에서 시각장애 편의시설을 이용하여 자발적으로 선진화된 시각장애 복지정책을 배워왔었습니다. 장애를 가지고 있다고 하여 자신의 장애에 구애받지 않는 직무만을 하려다보니 자신이 가지고 있는 능력을 펼치지 못하는 사람들이 많다고 생각합니다. 히든그레이스는 이러한 사람들에게 재능을 찾아주고 그 재능을 꽃피우게 하기 위해 교육을 실시하는 사회적 가치를 정확히 실현하는 사회적기업이라고 생각합니다.'}, {'id': 8, 'label': 'Q2', 'data': '히든그레이스에서 논문리뷰와 검증, 통계 교육 등 유의미한 결과물이 정말로 많은데 그 중에서 히든그레이스를 대표하는 프로젝트에 대한 결과물도 함께 있었으면 좋겠습니다. 만약 함께 일하게 된다면 머신러닝 프로젝트나 텍스트마이닝 등을 이용해 사회적약자에 대한 새로운 인사이트를 발굴해낼 수 있다고 생각합니다.'}, {'id': 9, 'label': 'Q3', 'data': '아직 채용이 되지 않았고 일해보지 않아서 단정 짓기 어렵지만 명시된 계약기간인 6개월 정도를 일해보고자 합니다. 6개월이라는 짧은 근무기간을 잡은 이유는 조금 더 깊은 전공 지식을 위해 대학원 진학을 희망하고 있기 때문입니다. '}]}

In [30]:
# 전체 자소서 가져와서 dataframe 만들기
url = 'http://localhost:3100/api/v1/'
cnt = 1
new_df = pd.DataFrame(columns=['id', 'name', 'q_1', 'q_2', 'q_3'])

while True:
    try:
        get_data ={
        'userId':cnt,
        'include':[
            'cosine'
            ]
        }
        header = {'Content-Type': 'application/json'}
        get_res =requests.post(url+'user', json.dumps(get_data), headers=header)

        result = get_res.json()

        userId = result['result']['user'][0]['userId']
        name = result['result']['user'][0]['lastName'] + result['result']['user'][0]['firstName']
        q_1 = result['result']['user'][0]['cosine']['questions'][0]['data']
        q_2 = result['result']['user'][0]['cosine']['questions'][1]['data']
        q_3 = result['result']['user'][0]['cosine']['questions'][2]['data']

        cos_df = pd.DataFrame({
            'id' : [userId],
            'name' : [name],
            'q_1' : [q_1],
            'q_2' : [q_2],
            'q_3' : [q_3]
        })
        new_df = pd.concat([new_df, cos_df])

        cnt += 1
    except Exception as e:
        break 

list index out of range


In [31]:
new_df

,id,name,q_1,q_2,q_3
0,1,오탁근,재작년에 진행했던 장애청년드림팀을 통해 다양한 장애를 가진 팀원들을 만나보고 느낀 ...,"히든그레이스에서 논문리뷰와 검증, 통계 교육 등 유의미한 결과물이 정말로 많은데 그...",아직 채용이 되지 않았고 일해보지 않아서 단정 짓기 어렵지만 명시된 계약기간인 6개...
0,2,고수민,여러 군데 입사지원을 알아보다가 데이터분석이라는 새로운 키워드를 주제로 삼은 귀하의...,"데이터분석을 통해 논문을 쓸 수 있게 도와주는 컨설팅회사로써, 좋은 방향의 회사라고...","현재 BE개발로써 취업을 하고 싶은 마음이 크나, 실제로 업무를 해본 적이 없어 경..."
0,3,이태준,보통 프로그래밍이나 UI/UX를 담당하고있는 회사라고 생각합니다.,"전화응대 및 사무보조, 자료조사에도 응할 것이며, 보완해야 할 점은 계약직에서 정규...","10년정도 일할 계획이며, 일을 하는데 있어서 중간에 승진 하는 데에 차질이 없이 ..."
0,4,정찬일,"“전산자료 입력원 및 사무 보조원, 전산자료 입력원(DB·단순자료)” 업무로서 귀사...",회사가 요구하는 수준에 최선의 노력을 할 것임을 약속합니다. 제 나이도 있어 ‘오래...,데이터(통계) 분석과 논문컨설팅을 전문으로 하는 기업. 어느 학문분야이든 전문적인 ...
0,5,최현진,장애인이나 사회 취약계층의 재능을 데이터로 분석해서 전문가로 양성하는 사회적 기업이...,뉴스 검색 등 매체를 통하여서 이 회사를 판단 할 수는 없다고 생각합니다. 제가 히...,저는 짧게 일할 생각으로는 일을 하지 않습니다. 몸이 힘든 거 보다 사람에게 받는 ...
...,...,...,...,...,...
0,76,이윤효,0,0,0
0,77,김혜진,청각장애로서 저를 채용이 된다면 아주 오래도록 일을 하고 싶습니다. 이런 저를 뽑아...,반드시 보안해야 할 점은 청각장애로서 문의 전화 상담이라는 게 그런 전화통화를 안됩...,인터넷 검색하고 난 후 이제야 알아챘습니다. 알고 보니 박사 석사과정 학위에 관련된...
0,78,허수현,이 부분은 일정 기간 근무 후 말씀 드릴 수 있을 ...,업무 진행 시 동료들과 소통을 통해 의견 조율이 가능하...,사람을 우선시 하는 회사라고 생각합니다. 검색 중 유투브 ...
0,79,위승희,매체에서 드러나는 단편적인 모습을 통해 저로서는 보완해야 할 점까지 파악하기는 힘들...,이번 채용이 6개월 계약직을 뽑는다고 알고 있습니다. 오래 일하는 데 문제가 되는 ...,"다른 사람들이 보지 못하는 가능성을 보고 그것을 실현하고자 노력하는 회사, 또한 그..."


# 데이터 전처리

In [34]:
for col in new_df.columns[-3:]:
    new_df[col] = new_df[col].map(prepro_text)

In [40]:
new_df

,id,name,q_1,q_2,q_3
0,1,오탁근,재작년 진행 했던 장애 청년 드림팀 통해 다양한 장애 가진 팀원 만나 보고 느낀 점...,히든 그레이스 논문 리뷰 검증 통계 교육 등 유 의미 결과물 정말로 많은데 그 중 ...,아직 채용 되지 않았고 일 해보지 않아서 단정 짓기 어렵지만 명시 된 계약 기간 개...
0,2,고수민,여러 군데 입사 지원 알아보다가 데이터 분석 새로운 키 워드 주제 삼은 귀하 회사 ...,데이터 분석 통해 논문 쓸 수 있게 도와주는 컨설팅 회사 로써 좋은 방향 회사 생각...,현재 개발 로써 취업 하고 싶은 마음 크나 실제 업무 해본 적 없어 경력 없습니다 ...
0,3,이태준,보통 프로그래밍 를 담당 하고있는 회사 생각 합니다,전화 응대 및 사무 보조 자료 조사 응 할 것 보완 해야 할 점 계약 직 정규직 전...,년 정도 일 할 계획 일 하는데 있어서 중간 승진 하는 데 차질 없이 일 할 수 있...
0,4,정찬일,전산 자료 입력 및 사무 보조 전산 자료 입력 단순 자료 업무 로서 귀사 기여 하기...,회사 요구 하는 수준 최선 노력 할 것임 약속 합니다 제 나이 있어 오래 저 스스로...,데이터 통계 분석 논문컨설팅 전문 하는 기업 어느 학문 분야 전문 영역 들어서는 첫...
0,5,최현진,장애인 사회 취약 계층 재능 데이터 분석 해서 전문가 양성 하는 사회 기업 는걸 보...,뉴스 검색 등 매체 통 하여서 이 회사 판단 할 수 없다고 생각 합니다 제 히든 그...,저 짧게 일 할 생각 일 하지 않습니다 몸 힘든 거 보다 사람 받는 스트레스 가장 ...
...,...,...,...,...,...
0,76,이윤효,,,
0,77,김혜진,청각장애 로서 저 채용 된다면 아주 오래도록 일 하고 싶습니다 이런 저 뽑아 주신다...,반드시 보안 해야 할 점 청각장애 로서 문의 전화 상담 게 그런 전화통화 안됩니다 ...,인터넷 검색 난 후 이제야 알아챘습니다 알 보니 박사 석사과정 학위 관련 된 논문 ...
0,78,허수현,이 부분 일정 기간 근무 후 말씀 드릴 수 있을 것 같습니다,업무 진행 시 동료 소통 통해 의견 조율 가능하고 맡은 분야 성장할 수 있는 환경 ...,사람 우선 시 하는 회사 생각 합니다 검색 중 유투브 클립 대표 경 영 철학 접 이...
0,79,위승희,매체 드러나는 단편 모습 통해 저 로서 보완 해야 할 점 파악 하기는 힘들지만 대신...,이번 채용 개월 계약 직 뽑는다고 알 있습니다 오래 일 하는 데 문제 되는 점 없습...,다른 사람 보지 못 하는 가능성 보고 그것 실현 하고자 노력 하는 회사 또한 그리스...


# 코사인 랭킹 구하기

In [ ]:
# 코사인 유사도 넣을 dataframe 생성
cos_df = pd.DataFrame({
    'id' : new_df['id'].values,
    'name' : new_df['name'].values,
    })

# 코사인 유사도 추출 & merge
for col in new_df.columns[-3:]:
    
    # fit할 데이터
    fit_data = new_df[col]
    fit_data.index = new_df['id'].values

    # 불용어사전
    stop_words = ['하는', '있는', '하여', '합니다', '하게', '지원', '입니다', '입사', '하겠습니다', '위해', '되었습니다', '싶습니다', '가지', '생각', '통해',
                  '회사', '히든', '그레이스', '분야', '해야', '같습니다', '관련', '오래', '하는데', '요소', '되는', '된다면', '있습니다', '하지', '정도', '하고',
                  '있을', '만약', '어떤', '때문', '있다고']
    # count 벡터라이저
    count_vec = CountVectorizer(
        stop_words=stop_words,
        min_df=2
    )
    # 빈도분석 결과
    _word, _vec = count_analyze(fit_data, count_vec)
    # 코사인 유사도 
    cosine_df = cosine_extraction(count_vec, fit_data, _word)
    # merge
    cos_df = pd.merge(cos_df, cosine_df, on='id', how='left')

# 전체 평균 구하기
cos_df['mean'] = cos_df.iloc[:,2:].mean(axis=1)
# 랭킹 구하기
cos_df['rank'] = cos_df['mean'].rank(ascending=False).astype(np.int64)

cos_df.sort_values(by='rank', axis=0, ascending=True, inplace=True)
cos_df

In [44]:
cos_df

,id,name,q_1,q_2,q_3,mean,rank
79,80,홍재희,0.467244,0.491723,0.326599,0.428522,1
54,55,이미숙,0.496704,0.477600,0.298142,0.424149,2
28,29,이지영,0.565685,0.223607,0.351382,0.380225,3
6,7,곽다희,0.326377,0.344124,0.326377,0.332292,4
50,51,이승수,0.452602,0.241209,0.298807,0.330873,5
...,...,...,...,...,...,...,...
26,27,장윤미,0.000000,0.000000,0.000000,0.000000,60
55,56,최형만,0.000000,0.000000,0.000000,0.000000,60
56,57,유충민,0.000000,0.000000,0.000000,0.000000,60
58,59,홍현승,0.000000,0.000000,0.000000,0.000000,60


# 유사도 & 랭킹 업데이트

In [ ]:
for i in cos_df['id']:
    cos_sim = float(cos_df['mean'][cos_df['id']==i].values[0])
    rank = int(cos_df['rank'][cos_df['id']==i].values[0])
    update_data =[{
        "userId": i,
        "cosine": cos_sim,
        "rank": rank,
        }]
    header = {'Content-Type': 'application/json'}
    update_res = requests.put(url+'cosine/update', json.dumps(data), headers=header)
    print()